In [1]:
%load_ext autoreload
%autoreload 2
%load_ext rich

In [35]:
from PyUncertainNumber.UP.vertex import vertexMethod
from PyUncertainNumber.UP.performance_func import cantilever_beam_func, cantilever_beam_deflection, cantilever_beam_stress
from PyUncertainNumber.UC.uncertainNumber import UncertainNumber as UN
from PyUncertainNumber.UC.utils import UNEncoder, cd_root_dir, to_database
import numpy as np
import json

In [3]:
cd_root_dir()

current directory: /Users/lesliec/Documents/Github_repos/daws2


## test marco's `endpoints` implementation

- The implemention below is based on Marco's `endpoints` function. For another similar implementation of the VertexMethod by Ioanna, refer to the other file named "Test_vertex_method.ipynb".

#### inputs

In [4]:
y = UN(name='beam width', symbol='y', units='m', essence='interval', interval_initialisation=[0.145, 0.155])
L = UN(name='beam length', symbol='L', units='m', essence='interval', interval_initialisation=[9.95, 10.05])
I = UN(name='moment of inertia', symbol='I', units='m', essence='interval', interval_initialisation=[0.0003861591, 0.0005213425])
F = UN(name='vertical force', symbol='F', units='kN', essence='interval', interval_initialisation=[11, 37])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='interval', interval_initialisation=[200, 220])

#### outputs

In [5]:
deflection_bounds_e = UN.endpointsMethod(vars=['L', 'I', 'F', 'E'], func=cantilever_beam_deflection, name='deflection', symbol='D')
deflection_bounds_e

UncertainNumber(name='deflection', symbol='D', essence='interval', interval_initialisation=(np.float64(0.031491609802257306), np.float64(0.16209976503208137)))

In [6]:
stress_bounds_e = UN.endpointsMethod(vars=['y', 'L', 'I', 'F'], func=cantilever_beam_stress, name='stress', symbol='S')
stress_bounds_e

UncertainNumber(name='stress', symbol='S', essence='interval', interval_initialisation=(np.float64(30.44112076034468), np.float64(149.25648521554976)))

# serialisation into JSON file or into database


**steps**

1. get json representation
2. from json into database

#### into Json

In [10]:
# a list of UN objects
input_UN_data = [y, L, I, F, E]
output_UN_data = [deflection_bounds_e, stress_bounds_e]

In [9]:
# json dump to filesystem

with open('./results/cantiliver_ex_inputs_data.json', 'w') as fp:
    json.dump(input_UN_data, fp, cls=UNEncoder, indent=4)


def json_dump(style='concise/verbose')
    """ inform the user about which fields are available """
    pass

#### into database

In [11]:
from pymongo import MongoClient

In [16]:
# a verbose way to save into the database
# create a JSON file containing all the input UNs

inputs_json_rep = json.dumps(input_UN_data, cls=UNEncoder)

lb = json.loads(inputs_json_rep)

# client = MongoClient()
# db = client.cantilever_beam_ex

# collection
inputs = db.inputs

# insert documents
new_result = inputs.insert_many(lb)

client.close()

In [36]:
''' a streamlined func to save to a database '''

to_database(dict_list=output_UN_data, db_name='cantilever_beam_ex', col_name='outputs')

InsertManyResult([ObjectId('66ba1138a737572b48f62717'), ObjectId('66ba1138a737572b48f62718')], acknowledged=True)

# Pool

In [ ]:
# type(inputs_json_rep)

# some_dict = dict(inputs_json_rep)
# some_dict

In [ ]:
''' from a list of dict to a dict '''

# new_dict = {item['symbol']:item for item in lb}
# new_dict